Установка необходимых библиотек

In [ ]:
!pip install langchain langchain-community chromadb beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Сбор данных посредоством WebBaseLoader

In [ ]:
from langchain.schema import Document
from bs4 import BeautifulSoup
import requests
import re


class CustomWebLoader:
  """
  Класс CustomWebLoader предназначен для загрузки данных из веб-страниц.

  Атрибуты:
  ----------------
  urls : list
    Список URL-адресов веб-страниц, которые необходимо загрузить.

  user_agent : str
    Строка user agent для имитации запросов веб-страниц. По умолчанию используется Mozilla Firefox.

  Методы:
  ----------------
  load():
    Делает запросы к веб-страницам, извлекает из них данные, обрабатывает и возвращает список документов, содержащих текст из загруженных страниц.
  """

  def __init__(self, urls, user_agent=None):
    self.urls = urls
    # Заголовок для избежания блокировок запросов на сайты
    self.user_agent = user_agent or "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

  def load(self):
    documents = []
    headers = {'User-Agent': self.user_agent}
    for url in self.urls:
      response = requests.get(url, headers=headers)
      if response.status_code != 200:
        print(f"Не удалось загрузить страницу: {url}")
        continue

      # Парсинг данных со стрницы
      soup = BeautifulSoup(response.text, 'html.parser')

      # Удаление ненужных элементов страницы - ссылок, навигационных элементов и т.д.
      for tag in soup(['nav', 'header', 'footer', 'aside', 'form', 'script', 'style', 'noscript', 'iframe', 'a']):
        tag.decompose()

      # Извлечение текста из страницы
      text = soup.get_text(separator=" ").strip()

      # Добавление текста в список документов, если он не пустой
      if text:
        clean_text = re.sub(r'\n\s+', '\n', text) # удаление лишних сочетаний "перенос строки + пробел"
        documents.append(Document(page_content=clean_text, metadata={"source": url}))

    return documents

# Список статей по тематикам ИБ
URL = ['https://selectel.ru/blog/ddos-attacks/', # DDoS
       'https://www.kaspersky.ru/resource-center/definitions/what-is-cryptography', # Криптография
       'https://www.kaspersky.ru/resource-center/preemptive-safety/phishing-prevention-tips', # Фишинг
       'https://ru.wikipedia.org/wiki/Многофакторная_аутентификация', # MFA
       'https://www.cloud4y.ru/blog/what-is-information-security/', # ИБ
       'https://habr.com/ru/companies/varonis/articles/526632/' #MITM
       ]

# Инициализация объекта загрузчика
loader = CustomWebLoader(URL)
# получение текста из загруженных страниц
documents = loader.load()

Разбитие данных на чанки размером 256 с перекрытием 50

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Используется рекурсивное разбиение с целью сохранить конктекст
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
chunking = text_splitter.split_documents(documents)

Создание объекта эмбеддера для эмбеддинга каждого чанка для последующего создания вектороной БД.

Объект создается на основе модели sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 с HuggingFace, в связи с чем необходимо ввести access token для получения доступа.

Данная модель была выбрана так как имеет не очень большой размер (118M параметров) и поддерживает русский язык.

In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

import os
from getpass import getpass

# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" # использование мультиязычной модели для эмбеддинга, поддерживает русский язык
)

··········


Создание векторного хранилища Chroma

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(chunking,embeddings)

Создание цепочки с моделью Mistral-Nemo-Instrict-2407

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import warnings

# отключение вывода предупреждений
warnings.filterwarnings("ignore")

# Создание промт шаблона для модели
prompt_template = """
Ты эксперт по информационной безопасности. Используй следующий контекст для ответа на вопрос в конце.
Если ты не знаешь ответа, просто скажи, что не знаешь, не пытайся делать предположения. Отвечай только на русском

{context}

Вопрос: {question}
Ответ:
"""

prompt = PromptTemplate(template=prompt_template,input_variables=['context','question'])


# инициализация модели
model = HuggingFaceHub(repo_id="mistralai/Mistral-Nemo-Instruct-2407",
                       model_kwargs={"temperature":0.5,
                                     "max_new_tokens":512,
                                     "max_length":64
                                    })
# инициализация ретривера на основе векторного хранилища. Тип поиска - mmr, то есть максимальная предельная релевантность, k - указывает на количество релевантных документов в выдаче
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":4})

qa = RetrievalQA.from_chain_type(llm=model,retriever=retriever,chain_type="stuff", chain_type_kwargs={"prompt": prompt})

query = input('Введите ваш запрос: ')
response = qa(query) # ответ модели

retrieved_docs = retriever.get_relevant_documents(query) # получение релевантных документов
set_of_retrieved_docs = set([doc.metadata['source'] for doc in retrieved_docs]) # создание множества из ссылок используемых документов для удаления дубликатов

start_index = response['result'].find('Ответ:') # начало ответа модели, ищется для "отсекания" промт шаблона
print(response['result'][start_index:])

print("\nИсточники, использованные для ответа:")
for i,source in enumerate(set_of_retrieved_docs):
  print(f"{i+1}. {source}")

Введите ваш запрос: что такое атака человек посередине?
Ответ:
Атака "человек посередине" (англ. man-in-the-middle) — это тип атаки, при котором злоумышленник вмешивается в коммуникацию между двумя сторонами, например, пользователем и веб-сайтом, и перехватывает передаваемые данные. Злоумышленник может подменять веб-сайт фальшивой версией или использовать уязвимости в сети, чтобы перехватывать трафик. При этом пользователь может не подозревать о том, что его данные перехватываются.

Источники, использованные для ответа:
1. https://habr.com/ru/companies/varonis/articles/526632/
2. https://www.kaspersky.ru/resource-center/preemptive-safety/phishing-prevention-tips
